In [17]:
import os

import numpy as np
import nibabel as nib
from nibabel import load as load_nii
from operator import add

In [13]:
root_dir = '/mnt/F2F25460F2542ADD/MedicalAnalysis/DataSets/ISBI/train/'
t1_name='mprage.nii'
mask_name='mask1.nii'
patch_size=(32,32)

In [6]:
# Get the names of the images and load them and normalize images
subjects = [f for f in sorted(os.listdir(root_dir)) if os.path.isdir(os.path.join(root_dir, f))]
subjects = subjects[:2]
image_names = [os.path.join(root_dir, subject, t1_name) for subject in subjects]

images = [fix_shape(load_nii(name).get_data()) for name in image_names]
images_norm = [(im.astype(np.float32) - im[np.nonzero(im)].mean()) / im[np.nonzero(im)].std() for im in images]
# images_norm = [image * (256.0 / (np.max(image) - np.min(image))) for image in images]

# load labels
label_names = [os.path.join(root_dir, subject, mask_name) for subject in subjects]
labels = [fix_shape(load_nii(name).get_data()) for name in label_names]

# positive classes (not background) classes between 1 and 14
p_vox_coord_pos = [get_mask_voxels(np.logical_and(mask > 0, mask < 15)) for mask in labels]

/home/microsoft/Medical_Analysis/Code/Environments/medical_pytorch_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  
/home/microsoft/Medical_Analysis/Code/Environments/medical_pytorch_env/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if sys.path[0] == '':


In [29]:
image=images_norm[0]
centers=p_vox_coord_pos[0]
mode='axial'

# By doing this we allow pooling when using encoders/unets.
patches = []
list_of_tuples = all([isinstance(center, tuple) for center in centers])
#sizes_match = [len(center) == len(patch_size) for center in centers]

# select between axial / coronal / saggital  patches
if mode == 'axial':
    patch_size = (patch_size[0], patch_size[1], 1)
if mode == 'coronal':
    patch_size = (patch_size[0], 1, patch_size[1])
if mode == 'saggital':
    patch_size = (1, patch_size[0], patch_size[1])
    
patch_half = tuple([idx/2 for idx in patch_size])
new_centers = [map(add, center, patch_half) for center in centers]
padding = tuple((idx, size-idx) for idx, size in zip(patch_half, patch_size))

new_image = np.pad(image, padding, mode='constant', constant_values=0)
axial_x_pos_patches = [np.array(get_patches(image, centers, size, mode='axial')) for image, centers in
                           zip(images_norm, p_vox_coord_pos)]

TypeError: `pad_width` must be of integral type.

TypeError: `pad_width` must be of integral type.

In [ ]:
def load_patch_vectors(options, name, label_name, dir_name, size, random_state, balance_neg=True):
    """
    Generate all patch vectors for all subjects and one sequence (name). This is done for each image view (axial, coronal and axial)
    In subcortical brain tissue segmentation, I am extracting all positive class voxels (classes from 1 to 14) and the same number of
    negatives (background) voxels (class 15).

    Inputs:
    - name: T1 image name
    - label_name: label name
    - dir_name = absolute path of the database images
    - size: patch size [p1, p2]
    - random_state: random seed

    Outputs:
    - x_axial: a list containing all the selected patches for all images for the axial view [image_num, num_samples, p1 , p2]
    - y_axial: a list containing all the labels for all image patches (axial view) [image_num, num_samples, p1 , p2]
    ...
    - y_saggital, a list containing all the labels for all image patches (saggital view) [image_num, num_samples, p1 , p2]
    - vox_positions: voxel coordinates for each of the patches [image_num, x, y, z]
    - image names
    """

    

    
    

    

    
    
    axial_y_pos_patches = [np.array(get_patches(image, centers, size, mode='axial')) for image, centers in
                           zip(labels, p_vox_coord_pos)]
    cor_x_pos_patches = [np.array(get_patches(image, centers, size, mode='coronal')) for image, centers in
                         zip(images_norm, p_vox_coord_pos)]
    cor_y_pos_patches = [np.array(get_patches(image, centers, size, mode='coronal')) for image, centers in
                         zip(labels, p_vox_coord_pos)]
    sag_x_pos_patches = [np.array(get_patches(image, centers, size, mode='saggital')) for image, centers in
                         zip(images_norm, p_vox_coord_pos)]
    sag_y_pos_patches = [np.array(get_patches(image, centers, size, mode='saggital')) for image, centers in
                         zip(labels, p_vox_coord_pos)]

    if balance_neg:
        n_vox_coord_pos = [get_mask_voxels(mask == 15, size=len(p)) for mask, p in zip(labels, p_vox_coord_pos)]
    else:
        n_vox_coord_pos = [get_mask_voxels(mask == 15) for mask, p in zip(labels, p_vox_coord_pos)]

    axial_x_neg_patches = [np.array(get_patches(image, centers, size, mode='axial')) for image, centers in
                           zip(images_norm, n_vox_coord_pos)]
    axial_y_neg_patches = [np.array(get_patches(image, centers, size, mode='axial')) for image, centers in
                           zip(labels, n_vox_coord_pos)]
    cor_x_neg_patches = [np.array(get_patches(image, centers, size, mode='coronal')) for image, centers in
                         zip(images_norm, n_vox_coord_pos)]
    cor_y_neg_patches = [np.array(get_patches(image, centers, size, mode='coronal')) for image, centers in
                         zip(labels, n_vox_coord_pos)]
    sag_x_neg_patches = [np.array(get_patches(image, centers, size, mode='saggital')) for image, centers in
                         zip(images_norm, n_vox_coord_pos)]
    sag_y_neg_patches = [np.array(get_patches(image, centers, size, mode='saggital')) for image, centers in
                         zip(labels, n_vox_coord_pos)]

    x_axial = [np.concatenate([p1, p2]) for p1, p2 in zip(axial_x_pos_patches, axial_x_neg_patches)]
    y_axial = [np.concatenate([p1, p2]) for p1, p2 in zip(axial_y_pos_patches, axial_y_neg_patches)]
    x_cor = [np.concatenate([p1, p2]) for p1, p2 in zip(cor_x_pos_patches, cor_x_neg_patches)]
    y_cor = [np.concatenate([p1, p2]) for p1, p2 in zip(cor_y_pos_patches, cor_y_neg_patches)]
    x_sag = [np.concatenate([p1, p2]) for p1, p2 in zip(sag_x_pos_patches, sag_x_neg_patches)]
    y_sag = [np.concatenate([p1, p2]) for p1, p2 in zip(sag_y_pos_patches, sag_y_neg_patches)]
    vox_positions = [np.concatenate([p1, p2]) for p1, p2 in zip(p_vox_coord_pos, n_vox_coord_pos)]

    return x_axial, y_axial, x_cor, y_cor, x_sag, y_sag, vox_positions, image_names

In [27]:
def get_patches(image, centers, patch_size=(32, 32), mode = 'axial'):
    """
    Returns 2D patches of each of the image views (coronal, axial, sagittal), given a list of voxel coordinates (centers).

    Input: 
    - image: input image
    - centers: voxel coordinates
    - patch_size to generate the patches [p1, p2]
    - mode: 'axial', 'coronal' or 'saggital' to generate different view patches. 

    Output:
    - A list of patches for each voxel passed as input in "centers" --> [num_voxels, p1, p2]
    """

    # If the size has even numbers, the patch will be centered. If not, it will try to create an square almost centered.
    # By doing this we allow pooling when using encoders/unets.
    patches = []
    list_of_tuples = all([isinstance(center, tuple) for center in centers])
    #sizes_match = [len(center) == len(patch_size) for center in centers]
    
    # select between axial / coronal / saggital  patches
    if mode == 'axial':
        patch_size = (patch_size[0], patch_size[1], 1)
    if mode == 'coronal':
        patch_size = (patch_size[0], 1, patch_size[1])
    if mode == 'saggital':
        patch_size = (1, patch_size[0], patch_size[1])

    patch_half = tuple([idx/2 for idx in patch_size])
    new_centers = [map(add, center, patch_half) for center in centers]
    padding = tuple((idx, size-idx) for idx, size in zip(patch_half, patch_size))
    new_image = np.pad(image, padding, mode='constant', constant_values=0)
    
    slices = [[slice(c_idx-p_idx, c_idx+(s_idx-p_idx)) for (c_idx, p_idx, s_idx) in zip(center, patch_half, patch_size)] for center in new_centers]
    patches = [np.squeeze(new_image[idx]) for idx in slices]

    return patches

In [26]:
np.squeeze?

In [ ]:
def load_patches(options, dir_name, mask_name, t1_name, size, seeds=None, balance_neg=True):
    """
    Load all patches for a given subject image passed as argument. This function makes no sense when using only
    one channel, but it's useful when using more than one, as load_patch_vectors is called for each of the channels and
    the outputs are stacked afterwards.

    input:
    - dir_name = absolute path of the database images
    - label_name: label name
    - t1_name: T1 image name
    - size: patch size [p1, p2]
    - seeds: list of images used as a seed

    output:
    - x_axial: a list containing all selected patches (axial view) [num_samples, p1, p2]
    - y_axial a list containing all selected labels (axial view) [num_samples, p1, p2]
    - ...
    - y_axial a list containing all selected labels (saggital view) [num_samples, p1, p2]
    - centers: voxel coordinates for each patch.
    """
    # Setting up the lists for all images

    random_state = np.random.randint(1)

    print '    --> Loading ' + t1_name + ' images'
    x_axial, y_axial, x_cor, y_cor, x_sag, y_sag, centers, t1_names = load_patch_vectors(options,
                                                                                         t1_name,
                                                                                         mask_name,
                                                                                         dir_name,
                                                                                         size,
                                                                                         random_state)
    # load atlas vectors
    x_atlas = get_atlas_vectors(options, dir_name, centers, t1_names)

    return x_axial, y_axial, x_cor, y_cor, x_sag, y_sag, x_atlas, t1_names

In [5]:
def get_mask_voxels(mask, size=None):
    """
    Return the voxel coordinates of non-zero voxels given a input image passed as argument.

    Input:
    - mask: Input image
    - size: if selected, return only a random list of length = size

    Output:
    - indices_list: a list of non-zero voxel positions expressed as a tuple [(x,y,z)]
    """
    
    import random
    indices = np.stack(np.nonzero(mask), axis=1)
    indices_list = [tuple(idx) for idx in indices]

    # if a output size is defined, shuffle and resize the list
    if size is not None:
        random.shuffle(indices_list)
        indices_list = indices_list[:size]

    return indices_list

In [4]:
def fix_shape(image):
    if isinstance(image, np.ndarray):
        if image.shape[-1] == 1:
            return np.squeeze(image)
    else:
        data = image.get_data()
        if data.shape[-1] == 1:
            data = np.squeeze(data)
        fixed = nib.Nifti1Image(data, image.affine)
        return fixed
    return image